* GroupBY()
* Pivot
* Window functions
* Sampling
* Greatest,Least,First,Last
* Collect_list
* String Functions- split,length,lower,upper,initcap,trim,padding(lpad,rpad),reverse,repeat,concat,concat_ws,substring,substring_index,instr,locate,translate,overlay

In [3]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql.functions import udf,col,length,expr,lit,lower,sumDistinct,sum,round,count,countDistinct,max,min,avg
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Train_hard").getOrCreate()

In [4]:
from pyspark.sql.window import Window
from pyspark.sql.window import *
from pyspark.sql.functions import *

In [2]:
data=[("James","Sales","NY",9000,34),
     ("Alicia","Sales","NY",8600,56),
     ("Robert","Sales","CA",8100,30),
     ("Lisa","Finance","CA",9000,24),
     ("Deja","Finance","CA",9900,40),
     ("Sugie","Finance","NY",8300,26),
     ("Ram","Finance","NY",7900,53),
     ("Kylie","Marketing","CA",8000,25),
     ("Reid","Marketing","NY",9100,50)]
schema=("Empname","dept","state","salary","age")
df=spark.createDataFrame(data,schema)
df.show()

+-------+---------+-----+------+---+
|Empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 26|
|    Ram|  Finance|   NY|  7900| 53|
|  Kylie|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



## Groupby

In [8]:
df.groupBy("dept").avg("age").alias("Avg_age").show()

+---------+--------+
|     dept|avg(age)|
+---------+--------+
|    Sales|    40.0|
|  Finance|   35.75|
|Marketing|    37.5|
+---------+--------+



In [16]:
nd=df.groupBy("dept").avg("age").alias("Avg_age")
nd.show()

+---------+--------+
|     dept|avg(age)|
+---------+--------+
|    Sales|    40.0|
|  Finance|   35.75|
|Marketing|    37.5|
+---------+--------+



In [17]:
nd.where(col("avg(age)")>38).show()

+-----+--------+
| dept|avg(age)|
+-----+--------+
|Sales|    40.0|
+-----+--------+



## Min

In [24]:
df.groupBy("dept","state").min("salary","age").show()

+---------+-----+-----------+--------+
|     dept|state|min(salary)|min(age)|
+---------+-----+-----------+--------+
|    Sales|   NY|       8600|      34|
|    Sales|   CA|       8100|      30|
|  Finance|   CA|       9000|      24|
|  Finance|   NY|       7900|      26|
|Marketing|   NY|       9100|      50|
|Marketing|   CA|       8000|      25|
+---------+-----+-----------+--------+



## Agg
* In case of applying multiple aggregate functions

In [32]:
df.groupBy("dept").agg(min("salary").alias("Min_sal"), \
                       avg("age").alias("Avg_age"),\
                      max("age").alias("Max_age")).show()

+---------+-------+-------+-------+
|     dept|Min_sal|Avg_age|Max_age|
+---------+-------+-------+-------+
|    Sales|   8100|   40.0|     56|
|  Finance|   7900|  35.75|     53|
|Marketing|   8000|   37.5|     50|
+---------+-------+-------+-------+



## Where with aggregate groupBY() clause

In [33]:
df.where(df.state=="NY").groupBy("dept").agg(min("salary").alias("Min_sal"), \
                       avg("age").alias("Avg_age"),\
                      max("age").alias("Max_age")).show()

+---------+-------+-------+-------+
|     dept|Min_sal|Avg_age|Max_age|
+---------+-------+-------+-------+
|    Sales|   8600|   45.0|     56|
|  Finance|   7900|   39.5|     53|
|Marketing|   9100|   50.0|     50|
+---------+-------+-------+-------+



In [36]:
df.groupBy("dept").agg(min("salary").alias("Min_sal"), \
                       avg("age").alias("Avg_age"),\
                      max("age").alias("Max_age")).where(col("Avg_age") > 38).show()

+-----+-------+-------+-------+
| dept|Min_sal|Avg_age|Max_age|
+-----+-------+-------+-------+
|Sales|   8100|   40.0|     56|
+-----+-------+-------+-------+



## Pivot

In [37]:
df1=df.select("dept","state","salary")
df1.show()

+---------+-----+------+
|     dept|state|salary|
+---------+-----+------+
|    Sales|   NY|  9000|
|    Sales|   NY|  8600|
|    Sales|   CA|  8100|
|  Finance|   CA|  9000|
|  Finance|   CA|  9900|
|  Finance|   NY|  8300|
|  Finance|   NY|  7900|
|Marketing|   CA|  8000|
|Marketing|   NY|  9100|
+---------+-----+------+



In [38]:
df1=df1.groupBy("dept","state").sum("salary").alias("Salary")
df1.show()

+---------+-----+-----------+
|     dept|state|sum(salary)|
+---------+-----+-----------+
|    Sales|   NY|      17600|
|    Sales|   CA|       8100|
|  Finance|   CA|      18900|
|  Finance|   NY|      16200|
|Marketing|   NY|       9100|
|Marketing|   CA|       8000|
+---------+-----+-----------+



In [51]:
pi=df1.groupBy("dept").pivot("state").sum("sum(salary)")
pi.show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



## Unpivot
* we dont have unpivot in spark, for that we have to use stack whic is a part of stack_SQL
#### Stack Example

In [43]:
spark.sql(""" select stack(3, 1,2,3,4,5,6)""").show()

+----+----+
|col0|col1|
+----+----+
|   1|   2|
|   3|   4|
|   5|   6|
+----+----+



* Since we pass 3 so (1,2,3,4,5,6) was conveted to 3 rows

In [57]:
pi.createOrReplaceTempView('tab')

spark.sql("""select * from tab""").show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



In [59]:
spark.sql("""select dept,stack(2,"California",CA,"NewYork",NY) as (State,Salary) from tab""").show()

+---------+----------+------+
|     dept|     State|Salary|
+---------+----------+------+
|    Sales|California|  8100|
|    Sales|   NewYork| 17600|
|  Finance|California| 18900|
|  Finance|   NewYork| 16200|
|Marketing|California|  8000|
|Marketing|   NewYork|  9100|
+---------+----------+------+



In [60]:
## upper table is similar to
df1.show()

+---------+-----+-----------+
|     dept|state|sum(salary)|
+---------+-----+-----------+
|    Sales|   NY|      17600|
|    Sales|   CA|       8100|
|  Finance|   CA|      18900|
|  Finance|   NY|      16200|
|Marketing|   NY|       9100|
|Marketing|   CA|       8000|
+---------+-----+-----------+



## Using selectExpr

In [61]:
## we dont touch dept as we want it as it is
pi.selectExpr("dept","stack(2,'California',CA,'NewYork',NY) as (State,Salary)").show()

+---------+----------+------+
|     dept|     State|Salary|
+---------+----------+------+
|    Sales|California|  8100|
|    Sales|   NewYork| 17600|
|  Finance|California| 18900|
|  Finance|   NewYork| 16200|
|Marketing|California|  8000|
|Marketing|   NewYork|  9100|
+---------+----------+------+



# Window API
* 3 types of window function
* 1. Ranking
* 2. Analytical
* 3. Aggregate

### Ranking- used to provide a ranking to the result after the partition

In [17]:
data=[("James","Sales","NY",9000,34),
     ("Alicia","Sales","NY",8600,56),
     ("Robert","Sales","CA",8100,30),
     ("John","Sales","AZ",8600,31),
     ("Ross","Sales","AZ",8100,33),
     ("Kathy","Sales","AZ",1000,39),
     ("Lisa","Finance","CA",9000,24),
     ("Deja","Finance","CA",9900,40),
     ("Sugie","Finance","NY",8300,26),
     ("Ram","Finance","NY",7900,53),
     ("Satya","Finance","AZ",8200,53),
     ("Kylie","Marketing","CA",8000,25),
     ("Reid","Marketing","NY",9100,50)]
schema=("Empname","dept","state","salary","age")
df=spark.createDataFrame(data,schema)
df.show()

+-------+---------+-----+------+---+
|Empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   John|    Sales|   AZ|  8600| 31|
|   Ross|    Sales|   AZ|  8100| 33|
|  Kathy|    Sales|   AZ|  1000| 39|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 26|
|    Ram|  Finance|   NY|  7900| 53|
|  Satya|  Finance|   AZ|  8200| 53|
|  Kylie|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



In [8]:
spec=Window.partitionBy("dept").orderBy(col("salary").desc())

In [18]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)).show()

+---------+------+---------------+
|     dept|salary|row_number_rank|
+---------+------+---------------+
|  Finance|  9900|              1|
|  Finance|  9000|              2|
|  Finance|  8300|              3|
|  Finance|  8200|              4|
|  Finance|  7900|              5|
|Marketing|  9100|              1|
|Marketing|  8000|              2|
|    Sales|  9000|              1|
|    Sales|  8600|              2|
|    Sales|  8600|              3|
|    Sales|  8100|              4|
|    Sales|  8100|              5|
|    Sales|  1000|              6|
+---------+------+---------------+



## Here note that 
* Sales|  8600|              2|
* Sales|  8600|              3| ---> are same but still we get number 2 and , this is beacuse of row number, tht's why we use Rank

## Rank

In [19]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)) \
.withColumn("Rank",rank().over(spec)).show()

+---------+------+---------------+----+
|     dept|salary|row_number_rank|Rank|
+---------+------+---------------+----+
|  Finance|  9900|              1|   1|
|  Finance|  9000|              2|   2|
|  Finance|  8300|              3|   3|
|  Finance|  8200|              4|   4|
|  Finance|  7900|              5|   5|
|Marketing|  9100|              1|   1|
|Marketing|  8000|              2|   2|
|    Sales|  9000|              1|   1|
|    Sales|  8600|              2|   2|
|    Sales|  8600|              3|   2|
|    Sales|  8100|              4|   4|
|    Sales|  8100|              5|   4|
|    Sales|  1000|              6|   6|
+---------+------+---------------+----+



## Note in here--
* dept|salary|row_number_rank|Rank|
* Sales|  8100|              5|   4|
* Sales|  1000|              6|   6|
* Here Rank from 4 goes to 6 coz it has 2 values of 4 so from 4 it goes directly to 6
* Thats why we use DEnse Rank

## Dense Rank

In [21]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)) \
.withColumn("Rank",rank().over(spec)) \
.withColumn("Dense_rank",dense_rank().over(spec)).show()

+---------+------+---------------+----+----------+
|     dept|salary|row_number_rank|Rank|Dense_rank|
+---------+------+---------------+----+----------+
|  Finance|  9900|              1|   1|         1|
|  Finance|  9000|              2|   2|         2|
|  Finance|  8300|              3|   3|         3|
|  Finance|  8200|              4|   4|         4|
|  Finance|  7900|              5|   5|         5|
|Marketing|  9100|              1|   1|         1|
|Marketing|  8000|              2|   2|         2|
|    Sales|  9000|              1|   1|         1|
|    Sales|  8600|              2|   2|         2|
|    Sales|  8600|              3|   2|         2|
|    Sales|  8100|              4|   4|         3|
|    Sales|  8100|              5|   4|         3|
|    Sales|  1000|              6|   6|         4|
+---------+------+---------------+----+----------+



# Percent_rank

In [22]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)) \
.withColumn("Rank",rank().over(spec)) \
.withColumn("Dense_rank",dense_rank().over(spec)) \
.withColumn("Percent_rank",percent_rank().over(spec)).show()

+---------+------+---------------+----+----------+------------+
|     dept|salary|row_number_rank|Rank|Dense_rank|Percent_rank|
+---------+------+---------------+----+----------+------------+
|  Finance|  9900|              1|   1|         1|         0.0|
|  Finance|  9000|              2|   2|         2|        0.25|
|  Finance|  8300|              3|   3|         3|         0.5|
|  Finance|  8200|              4|   4|         4|        0.75|
|  Finance|  7900|              5|   5|         5|         1.0|
|Marketing|  9100|              1|   1|         1|         0.0|
|Marketing|  8000|              2|   2|         2|         1.0|
|    Sales|  9000|              1|   1|         1|         0.0|
|    Sales|  8600|              2|   2|         2|         0.2|
|    Sales|  8600|              3|   2|         2|         0.2|
|    Sales|  8100|              4|   4|         3|         0.6|
|    Sales|  8100|              5|   4|         3|         0.6|
|    Sales|  1000|              6|   6| 

*      Sales|  9000|              1|   1|         1|         0.0|
* |    Sales|  8600|              2|   2|         2|         0.2|
* |    Sales|  8600|              3|   2|         2|         0.2|
* |    Sales|  8100|              4|   4|         3|         0.6|
* |    Sales|  8100|              5|   4|         3|         0.6|
* |    Sales|  1000|              6|   6|         4|         1.0|

* In here 1 is zero, now at 2--it is 2/5=.2 at num 3 
* as well since 8600 is same for 2 and 3 row num, at num 4--3/5=0.6

## Ntile()

In [25]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)) \
.withColumn("Rank",rank().over(spec)) \
.withColumn("Dense_rank",dense_rank().over(spec)) \
.withColumn("Percent_rank",percent_rank().over(spec)) \
.withColumn("Ntile",ntile(6).over(spec)).show()
## we pass 6 coz we know max variable i.e sales have 6 numbers-- so we pass 6 in NTILE(6)

+---------+------+---------------+----+----------+------------+-----+
|     dept|salary|row_number_rank|Rank|Dense_rank|Percent_rank|Ntile|
+---------+------+---------------+----+----------+------------+-----+
|  Finance|  9900|              1|   1|         1|         0.0|    1|
|  Finance|  9000|              2|   2|         2|        0.25|    2|
|  Finance|  8300|              3|   3|         3|         0.5|    3|
|  Finance|  8200|              4|   4|         4|        0.75|    4|
|  Finance|  7900|              5|   5|         5|         1.0|    5|
|Marketing|  9100|              1|   1|         1|         0.0|    1|
|Marketing|  8000|              2|   2|         2|         1.0|    2|
|    Sales|  9000|              1|   1|         1|         0.0|    1|
|    Sales|  8600|              2|   2|         2|         0.2|    2|
|    Sales|  8600|              3|   2|         2|         0.2|    3|
|    Sales|  8100|              4|   4|         3|         0.6|    4|
|    Sales|  8100|  

* If we pass n=3 then

In [26]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)) \
.withColumn("Rank",rank().over(spec)) \
.withColumn("Dense_rank",dense_rank().over(spec)) \
.withColumn("Percent_rank",percent_rank().over(spec)) \
.withColumn("Ntile",ntile(3).over(spec)).show()

+---------+------+---------------+----+----------+------------+-----+
|     dept|salary|row_number_rank|Rank|Dense_rank|Percent_rank|Ntile|
+---------+------+---------------+----+----------+------------+-----+
|  Finance|  9900|              1|   1|         1|         0.0|    1|
|  Finance|  9000|              2|   2|         2|        0.25|    1|
|  Finance|  8300|              3|   3|         3|         0.5|    2|
|  Finance|  8200|              4|   4|         4|        0.75|    2|
|  Finance|  7900|              5|   5|         5|         1.0|    3|
|Marketing|  9100|              1|   1|         1|         0.0|    1|
|Marketing|  8000|              2|   2|         2|         1.0|    2|
|    Sales|  9000|              1|   1|         1|         0.0|    1|
|    Sales|  8600|              2|   2|         2|         0.2|    1|
|    Sales|  8600|              3|   2|         2|         0.2|    2|
|    Sales|  8100|              4|   4|         3|         0.6|    2|
|    Sales|  8100|  

## Cumulative distribution

In [27]:
df.select("dept","salary").withColumn("row_number_rank",row_number().over(spec)) \
.withColumn("Rank",rank().over(spec)) \
.withColumn("Dense_rank",dense_rank().over(spec)) \
.withColumn("Percent_rank",percent_rank().over(spec)) \
.withColumn("Ntile",ntile(3).over(spec)) \
.withColumn("Cumu_dist",cume_dist().over(spec)).show()

+---------+------+---------------+----+----------+------------+-----+-------------------+
|     dept|salary|row_number_rank|Rank|Dense_rank|Percent_rank|Ntile|          Cumu_dist|
+---------+------+---------------+----+----------+------------+-----+-------------------+
|  Finance|  9900|              1|   1|         1|         0.0|    1|                0.2|
|  Finance|  9000|              2|   2|         2|        0.25|    1|                0.4|
|  Finance|  8300|              3|   3|         3|         0.5|    2|                0.6|
|  Finance|  8200|              4|   4|         4|        0.75|    2|                0.8|
|  Finance|  7900|              5|   5|         5|         1.0|    3|                1.0|
|Marketing|  9100|              1|   1|         1|         0.0|    1|                0.5|
|Marketing|  8000|              2|   2|         2|         1.0|    2|                1.0|
|    Sales|  9000|              1|   1|         1|         0.0|    1|0.16666666666666666|
|    Sales

In [41]:
print(1/6)
print("-"*100)
print(5/6)
print("-"*100)
print("We get 0.5 because we do 3/5 coz 9000 is 1 nd 86000 ciunt to 2 and 3 so its 3/6 i.e {}".format(3/6))
print("-"*100)
print("Similarly we get 5/6-{}".format(5/6))

0.16666666666666666
----------------------------------------------------------------------------------------------------
0.8333333333333334
----------------------------------------------------------------------------------------------------
We get 0.5 because we do 3/5 coz 9000 is 1 nd 86000 ciunt to 2 and 3 so its 3/6 i.e 0.5
----------------------------------------------------------------------------------------------------
Similarly we get 5/6-0.8333333333333334


In [35]:
5/6

0.8333333333333334

## Analytical Windows

##### 1)-Lag-take 3 arguments("col_name",lag_value,"value when lag is not there")

In [5]:
spec=Window.partitionBy("dept").orderBy("salary")

In [7]:
df.select("dept","salary").withColumn("lag_column",lag("salary",1,-1).over(spec)).show()

+---------+------+----------+
|     dept|salary|lag_column|
+---------+------+----------+
|  Finance|  7900|        -1|
|  Finance|  8300|      7900|
|  Finance|  9000|      8300|
|  Finance|  9900|      9000|
|Marketing|  8000|        -1|
|Marketing|  9100|      8000|
|    Sales|  8100|        -1|
|    Sales|  8600|      8100|
|    Sales|  9000|      8600|
+---------+------+----------+



## Check above
* where there is no lag value its replaced by -1, and lag followed by 1 row each

#### lead

In [9]:
df.select("dept","salary").withColumn("lag_column",lag("salary",1,-1).over(spec)) \
.withColumn("lead_column",lead("salary",1,-1).over(spec)).show()

+---------+------+----------+-----------+
|     dept|salary|lag_column|lead_column|
+---------+------+----------+-----------+
|  Finance|  7900|        -1|       8300|
|  Finance|  8300|      7900|       9000|
|  Finance|  9000|      8300|       9900|
|  Finance|  9900|      9000|         -1|
|Marketing|  8000|        -1|       9100|
|Marketing|  9100|      8000|         -1|
|    Sales|  8100|        -1|       8600|
|    Sales|  8600|      8100|       9000|
|    Sales|  9000|      8600|         -1|
+---------+------+----------+-----------+



## Analysis window

* Q- Print sum of salary according to each department ?

In [12]:
data=[("James","Sales","NY",9000,34),
     ("Alicia","Sales","NY",8600,56),
     ("Robert","Sales","CA",8100,30),
     ("John","Sales","AZ",8600,31),
     ("Ross","Sales","AZ",8100,33),
     ("Kathy","Sales","AZ",1000,39),
     ("Lisa","Finance","CA",9000,24),
     ("Deja","Finance","CA",9900,40),
     ("Sugie","Finance","NY",8300,26),
     ("Ram","Finance","NY",7900,53),
     ("Satya","Finance","AZ",8200,53),
     ("Kylie","Marketing","CA",8000,25),
     ("Reid","Marketing","NY",9100,50)]
schema=("Empname","dept","state","salary","age")
df=spark.createDataFrame(data,schema)
df.show(3)

+-------+-----+-----+------+---+
|Empname| dept|state|salary|age|
+-------+-----+-----+------+---+
|  James|Sales|   NY|  9000| 34|
| Alicia|Sales|   NY|  8600| 56|
| Robert|Sales|   CA|  8100| 30|
+-------+-----+-----+------+---+
only showing top 3 rows



In [13]:
spec=Window.partitionBy("dept")
df.select("dept","salary").withColumn("Sum_salary",sum("Salary").over(spec)).show()

+---------+------+----------+
|     dept|salary|Sum_salary|
+---------+------+----------+
|  Finance|  9000|     43300|
|  Finance|  9900|     43300|
|  Finance|  8300|     43300|
|  Finance|  7900|     43300|
|  Finance|  8200|     43300|
|Marketing|  8000|     17100|
|Marketing|  9100|     17100|
|    Sales|  9000|     43400|
|    Sales|  8600|     43400|
|    Sales|  8100|     43400|
|    Sales|  8600|     43400|
|    Sales|  8100|     43400|
|    Sales|  1000|     43400|
+---------+------+----------+



* @- find out max salary from each dept using window funcctuon

In [23]:
spec=Window.partitionBy("dept").orderBy(col("Salary").desc())
df.select("dept","salary").withColumn("Max_salary",first("Salary").over(spec)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  9900|      9900|
|  Finance|  9000|      9900|
|  Finance|  8300|      9900|
|  Finance|  8200|      9900|
|  Finance|  7900|      9900|
|Marketing|  9100|      9100|
|Marketing|  8000|      9100|
|    Sales|  9000|      9000|
|    Sales|  8600|      9000|
|    Sales|  8600|      9000|
|    Sales|  8100|      9000|
|    Sales|  8100|      9000|
|    Sales|  1000|      9000|
+---------+------+----------+



## Range_between() and row_between()

In [25]:
spec=Window.partitionBy("dept").orderBy(df.salary). \
rowsBetween(Window.currentRow,Window.unboundedFollowing)

In [26]:
df.select("dept","salary").withColumn("Max_salary",sum("Salary").over(spec)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  7900|     43300|
|  Finance|  8200|     35400|
|  Finance|  8300|     27200|
|  Finance|  9000|     18900|
|  Finance|  9900|      9900|
|Marketing|  8000|     17100|
|Marketing|  9100|      9100|
|    Sales|  1000|     43400|
|    Sales|  8100|     42400|
|    Sales|  8100|     34300|
|    Sales|  8600|     26200|
|    Sales|  8600|     17600|
|    Sales|  9000|      9000|
+---------+------+----------+



In [31]:
## Here
print("In finanace 9900+9000= {}".format(9900+9000))
print("-"*100)
print("In finanace 9900+9000+8300 = {}".format(9900+9000+8300))
print("-"*100)

In finanace 9900+9000= 18900
----------------------------------------------------------------------------------------------------
In finanace 9900+9000+8300 = 27200
----------------------------------------------------------------------------------------------------


## range Between

In [36]:
spec1=Window.partitionBy("dept").orderBy(df.salary).rangeBetween(Window.currentRow,Window.unboundedFollowing)
df.select("dept","salary").withColumn("Max_salary",sum("Salary").over(spec1)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  7900|     43300|
|  Finance|  8200|     35400|
|  Finance|  8300|     27200|
|  Finance|  9000|     18900|
|  Finance|  9900|      9900|
|Marketing|  8000|     17100|
|Marketing|  9100|      9100|
|    Sales|  1000|     43400|
|    Sales|  8100|     42400|
|    Sales|  8100|     42400|
|    Sales|  8600|     26200|
|    Sales|  8600|     26200|
|    Sales|  9000|      9000|
+---------+------+----------+



### using (indow.unboundedPreceding),indow.unboundedFollowing)

In [37]:
spec1=Window.partitionBy("dept").orderBy(df.salary).rangeBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.select("dept","salary").withColumn("Max_salary",sum("Salary").over(spec1)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  7900|     43300|
|  Finance|  8200|     43300|
|  Finance|  8300|     43300|
|  Finance|  9000|     43300|
|  Finance|  9900|     43300|
|Marketing|  8000|     17100|
|Marketing|  9100|     17100|
|    Sales|  1000|     43400|
|    Sales|  8100|     43400|
|    Sales|  8100|     43400|
|    Sales|  8600|     43400|
|    Sales|  8600|     43400|
|    Sales|  9000|     43400|
+---------+------+----------+



## Difference between rowBetween and rangebetween

* rangeBetween

In [46]:
spec1=Window.partitionBy("dept").orderBy(df.salary).rangeBetween(Window.currentRow,500)
df.select("dept","salary").withColumn("Max_salary",sum("Salary").over(spec1)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  7900|     24400|
|  Finance|  8200|     16500|
|  Finance|  8300|      8300|
|  Finance|  9000|      9000|
|  Finance|  9900|      9900|
|Marketing|  8000|      8000|
|Marketing|  9100|      9100|
|    Sales|  1000|      1000|
|    Sales|  8100|     33400|
|    Sales|  8100|     33400|
|    Sales|  8600|     26200|
|    Sales|  8600|     26200|
|    Sales|  9000|      9000|
+---------+------+----------+



In [51]:
print(" SInce we define the boundry to 7900+500=8400, so its qualfiled till 8300 boundry to do the sum {}".format(7900+8200+8300))
print("-"*100)
print("Similarly 8200+500=8700, so its qualfiled till 8300 boundry to do the sum-{}".format(8200+8300))

 SInce we define the boundry to 7900+500=8400, so its qualfiled till 8300 boundry to do the sum 24400
----------------------------------------------------------------------------------------------------
Similarly 8200+500=8700, so its qualfiled till 8300 boundry to do the sum-16500


* rowsbetween()

In [54]:
spec1=Window.partitionBy("dept").orderBy(df.salary).rowsBetween(Window.currentRow,500)
df.select("dept","salary").withColumn("Max_salary",sum("Salary").over(spec1)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  7900|     43300|
|  Finance|  8200|     35400|
|  Finance|  8300|     27200|
|  Finance|  9000|     18900|
|  Finance|  9900|      9900|
|Marketing|  8000|     17100|
|Marketing|  9100|      9100|
|    Sales|  1000|     43400|
|    Sales|  8100|     42400|
|    Sales|  8100|     34300|
|    Sales|  8600|     26200|
|    Sales|  8600|     17600|
|    Sales|  9000|      9000|
+---------+------+----------+



## Nothing happended it just simply dod the sum

In [55]:
spec1=Window.partitionBy("dept").orderBy(df.salary).rowsBetween(Window.currentRow,2)
df.select("dept","salary").withColumn("Max_salary",sum("Salary").over(spec1)).show()

+---------+------+----------+
|     dept|salary|Max_salary|
+---------+------+----------+
|  Finance|  7900|     24400|
|  Finance|  8200|     25500|
|  Finance|  8300|     27200|
|  Finance|  9000|     18900|
|  Finance|  9900|      9900|
|Marketing|  8000|     17100|
|Marketing|  9100|      9100|
|    Sales|  1000|     17200|
|    Sales|  8100|     24800|
|    Sales|  8100|     25300|
|    Sales|  8600|     26200|
|    Sales|  8600|     17600|
|    Sales|  9000|      9000|
+---------+------+----------+



 * In here its simply adding the do preceding rows together

## Sampling

In [3]:
df= spark.range(100)
df.show(5)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [6]:
df.sample(fraction=0.2).show() ## Showing 20% of data as we pass frction=0.2

+---+
| id|
+---+
|  0|
|  9|
| 10|
| 15|
| 19|
| 25|
| 33|
| 46|
| 51|
| 52|
| 57|
| 60|
| 78|
| 80|
| 86|
| 87|
| 89|
+---+



In [8]:
df.sample(fraction=0.2,withReplacement=True).show() ## withReplacement=False means every elemt is unique,if taken out its not placed back
                                                    #withReplacement True we are allowing dublicate samples

+---+
| id|
+---+
| 10|
| 12|
| 17|
| 18|
| 23|
| 23|
| 35|
| 37|
| 39|
| 42|
| 59|
| 74|
| 77|
| 80|
| 84|
| 96|
| 98|
+---+



In [11]:
## Using Seed--it gives same data everytme
df.sample(fraction=0.2,withReplacement=True,seed=10).show()

+---+
| id|
+---+
|  1|
|  6|
|  8|
| 17|
| 30|
| 31|
| 42|
| 45|
| 52|
| 60|
| 61|
| 63|
| 68|
| 69|
| 73|
| 75|
| 75|
| 79|
| 83|
| 86|
+---+
only showing top 20 rows



## SelectBy

In [13]:
df=spark.range(100).select((col("id") % 3).alias("Key"))
df.show()

+---+
|Key|
+---+
|  0|
|  1|
|  2|
|  0|
|  1|
|  2|
|  0|
|  1|
|  2|
|  0|
|  1|
|  2|
|  0|
|  1|
|  2|
|  0|
|  1|
|  2|
|  0|
|  1|
+---+
only showing top 20 rows



In [18]:
df.distinct().show()

+---+
|Key|
+---+
|  0|
|  1|
|  2|
+---+



#### If we want 10% of zero, 40% of one then we use SampleBY

In [19]:
sample=df.sampleBy("Key",fractions={0:0.1,1:0.4},seed=20)
sample.show()

+---+
|Key|
+---+
|  1|
|  0|
|  1|
|  1|
|  0|
|  1|
|  1|
|  1|
|  0|
|  1|
|  0|
|  1|
|  1|
|  0|
|  1|
|  1|
|  1|
|  1|
|  1|
+---+



In [20]:
sample.groupBy("Key").count().show()

+---+-----+
|Key|count|
+---+-----+
|  0|    5|
|  1|   14|
+---+-----+



## Other aggregate fumctions

In [4]:
data=[("James","Sales","NY",None,34),
     ("Alicia","Sales","NY",8600,56),
     ("Robert","Sales","CA",8100,30),
     ("John","Sales","AZ",8600,31),
     ("Ross","Sales","AZ",8100,33),
     ("Kathy","Sales","AZ",1000,39),
     ("Lisa","Finance","CA",9000,24),
     ("Deja","Finance","CA",9900,40),
     ("Sugie","Finance","NY",8300,26),
     ("Ram","Finance","NY",7900,53),
     ("Satya","Finance","AZ",8200,53),
     ("Kylie","Marketing","CA",8000,25),
     ("Reid","Marketing","NY",9100,50)]
schema=("Empname","dept","state","salary","age")
df=spark.createDataFrame(data,schema)
df.show()

+-------+---------+-----+------+---+
|Empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  null| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   John|    Sales|   AZ|  8600| 31|
|   Ross|    Sales|   AZ|  8100| 33|
|  Kathy|    Sales|   AZ|  1000| 39|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 26|
|    Ram|  Finance|   NY|  7900| 53|
|  Satya|  Finance|   AZ|  8200| 53|
|  Kylie|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



### first

In [5]:
df.select(first(df.salary)).show()

+-------------+
|first(salary)|
+-------------+
|         null|
+-------------+



In [6]:
## If we dont want nulll then
df.select(first(df.salary,ignorenulls=True)).show()

+-------------+
|first(salary)|
+-------------+
|         8600|
+-------------+



#### last

In [7]:
df.select(last(df.salary)).show()

+------------+
|last(salary)|
+------------+
|        9100|
+------------+



### Greatest

In [9]:
df.select(greatest(df.age,df.salary)).show()

+---------------------+
|greatest(age, salary)|
+---------------------+
|                   34|
|                 8600|
|                 8100|
|                 8600|
|                 8100|
|                 1000|
|                 9000|
|                 9900|
|                 8300|
|                 7900|
|                 8200|
|                 8000|
|                 9100|
+---------------------+



#### Least

In [10]:
df.select(least(df.age,df.salary)).show()

+------------------+
|least(age, salary)|
+------------------+
|                34|
|                56|
|                30|
|                31|
|                33|
|                39|
|                24|
|                40|
|                26|
|                53|
|                53|
|                25|
|                50|
+------------------+



### Collect_list--> allow duplicate values as well eg-53 in below example

In [13]:
df.select(collect_list(df.age)).show(truncate=False)

+----------------------------------------------------+
|collect_list(age)                                   |
+----------------------------------------------------+
|[34, 56, 30, 31, 33, 39, 24, 40, 26, 53, 53, 25, 50]|
+----------------------------------------------------+



### Collect_Set--> don't allow duplicate values eg-53 only come one time

In [15]:
df.select(collect_set(df.age)).show(truncate=False)

+------------------------------------------------+
|collect_set(age)                                |
+------------------------------------------------+
|[33, 30, 34, 31, 56, 53, 50, 24, 39, 25, 40, 26]|
+------------------------------------------------+



## Column Functions
* 1- Concat()

In [19]:
df.select(concat(col("salary"),lit("|"),col("age")).alias("Concat_col")).show()

+----------+
|Concat_col|
+----------+
|      null|
|   8600|56|
|   8100|30|
|   8600|31|
|   8100|33|
|   1000|39|
|   9000|24|
|   9900|40|
|   8300|26|
|   7900|53|
|   8200|53|
|   8000|25|
|   9100|50|
+----------+



##### If age <30 "KID", age >30 and <45 "Mid_age" more than 45 OLD

In [25]:
df.withColumn("Status",when(col("age")<30,"KID").\
               when((df.age>30) & (df.age <45),"Adults").\
              otherwise("Men")).show()

+-------+---------+-----+------+---+------+
|Empname|     dept|state|salary|age|Status|
+-------+---------+-----+------+---+------+
|  James|    Sales|   NY|  null| 34|Adults|
| Alicia|    Sales|   NY|  8600| 56|   Men|
| Robert|    Sales|   CA|  8100| 30|   Men|
|   John|    Sales|   AZ|  8600| 31|Adults|
|   Ross|    Sales|   AZ|  8100| 33|Adults|
|  Kathy|    Sales|   AZ|  1000| 39|Adults|
|   Lisa|  Finance|   CA|  9000| 24|   KID|
|   Deja|  Finance|   CA|  9900| 40|Adults|
|  Sugie|  Finance|   NY|  8300| 26|   KID|
|    Ram|  Finance|   NY|  7900| 53|   Men|
|  Satya|  Finance|   AZ|  8200| 53|   Men|
|  Kylie|Marketing|   CA|  8000| 25|   KID|
|   Reid|Marketing|   NY|  9100| 50|   Men|
+-------+---------+-----+------+---+------+



* Parttions()

In [29]:
df1=spark.range(10)
df1.rdd.getNumPartitions()

4

##### Doing re_partitions

In [30]:
df1=df1.repartition(5)
df1.rdd.getNumPartitions()

5

### Selecting the ID'S

In [31]:
df1.select("ID",spark_partition_id()).show()

+---+--------------------+
| ID|SPARK_PARTITION_ID()|
+---+--------------------+
|  6|                   0|
|  7|                   0|
|  0|                   1|
|  3|                   1|
|  8|                   1|
|  1|                   2|
|  2|                   2|
|  9|                   2|
|  4|                   3|
|  5|                   4|
+---+--------------------+



#### Here id's (6-7) belongs to ID ZERO, Id's (0,3,8) belongs to ID ONE.

## String Functions

##### split

In [9]:
df=spark.read.load('orders/part-00000',format="csv",schema='order_id int,order_date timestamp,order_customer_id int, \
                   order_status string')
df.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [9]:
dd = spark.createDataFrame([('oneAtwoBthreeC',)], ['s',])
dd.select(split(dd.s, '[ABC]', 1).alias('s')).show()

+----------------+
|               s|
+----------------+
|[oneAtwoBthreeC]|
+----------------+



In [18]:
df.withColumn("Year",split(df.order_date,"-")).show(2)

+--------+-------------------+-----------------+---------------+--------------------+
|order_id|         order_date|order_customer_id|   order_status|                Year|
+--------+-------------------+-----------------+---------------+--------------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|[2013, 07, 25 00:...|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|[2013, 07, 25 00:...|
+--------+-------------------+-----------------+---------------+--------------------+
only showing top 2 rows



In [17]:
df.withColumn("Year",split(df.order_date,"-")[0]).show(2)

+--------+-------------------+-----------------+---------------+----+
|order_id|         order_date|order_customer_id|   order_status|Year|
+--------+-------------------+-----------------+---------------+----+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|2013|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|2013|
+--------+-------------------+-----------------+---------------+----+
only showing top 2 rows



### Split using regular expressions

In [24]:
df=spark.createDataFrame([("ab12cd23fe27kl",)],["col",])
df.show()

+--------------+
|           col|
+--------------+
|ab12cd23fe27kl|
+--------------+



In [26]:
df.select(split(df.col,'[0-9]+')).show()

+----------------------+
|split(col, [0-9]+, -1)|
+----------------------+
|      [ab, cd, fe, kl]|
+----------------------+



#### Length

In [31]:
df.select(col("order_status"),length(col("order_status")).alias("staus_length")).show(5)

+---------------+------------+
|   order_status|staus_length|
+---------------+------------+
|         CLOSED|           6|
|PENDING_PAYMENT|          15|
|       COMPLETE|           8|
|         CLOSED|           6|
|       COMPLETE|           8|
+---------------+------------+
only showing top 5 rows



#### lower()

In [37]:
df1=df.withColumn("lower_stat",lower(col("order_status")))
df1.show(2)

+--------+-------------------+-----------------+---------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|     lower_stat|
+--------+-------------------+-----------------+---------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|         closed|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|pending_payment|
+--------+-------------------+-----------------+---------------+---------------+
only showing top 2 rows



In [41]:
df1.select(col("lower_stat"),initcap(col("order_status"))).show(3)

+---------------+---------------------+
|     lower_stat|initcap(order_status)|
+---------------+---------------------+
|         closed|               Closed|
|pending_payment|      Pending_payment|
|       complete|             Complete|
+---------------+---------------------+
only showing top 3 rows



### Trim

In [5]:
df=spark.createDataFrame([("          spark        ",),("          developer        ",)],schema=["col1"])
df.show(truncate=False)

+---------------------------+
|col1                       |
+---------------------------+
|          spark            |
|          developer        |
+---------------------------+



In [11]:
df.withColumn("trimmed_val",trim(col("col1"))).show()

+--------------------+-----------+
|                col1|trimmed_val|
+--------------------+-----------+
|          spark  ...|      spark|
|          develop...|  developer|
+--------------------+-----------+



### ltrim-- Trim Left

In [7]:
df.withColumn("trim_left",ltrim(col("col1"))).show(truncate=False)

+---------------------------+-----------------+
|col1                       |trim_left        |
+---------------------------+-----------------+
|          spark            |spark            |
|          developer        |developer        |
+---------------------------+-----------------+



#### rtrim--- Trim Right

In [8]:
df.withColumn("trim_right",rtrim(col("col1"))).show(truncate=False)

+---------------------------+-------------------+
|col1                       |trim_right         |
+---------------------------+-------------------+
|          spark            |          spark    |
|          developer        |          developer|
+---------------------------+-------------------+



## Padding

In [10]:
df.show(2)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
+--------+-------------------+-----------------+---------------+
only showing top 2 rows



### lpad-- Padding left

In [13]:
df.select(col("order_id"),lpad(df.order_id,5,"@").alias("padding_left")).show(5)

+--------+------------+
|order_id|padding_left|
+--------+------------+
|       1|       @@@@1|
|       2|       @@@@2|
|       3|       @@@@3|
|       4|       @@@@4|
|       5|       @@@@5|
+--------+------------+
only showing top 5 rows



### rpad-- Padding right

In [15]:
df.select(col("order_id"),rpad(df.order_id,5,"#").alias("padding_right")).show(5)

+--------+-------------+
|order_id|padding_right|
+--------+-------------+
|       1|        1####|
|       2|        2####|
|       3|        3####|
|       4|        4####|
|       5|        5####|
+--------+-------------+
only showing top 5 rows



## reverse

In [16]:
df.select(col("order_status"),reverse(df.order_status).alias("Reverse_string")).show(5)

+---------------+---------------+
|   order_status| Reverse_string|
+---------------+---------------+
|         CLOSED|         DESOLC|
|PENDING_PAYMENT|TNEMYAP_GNIDNEP|
|       COMPLETE|       ETELPMOC|
|         CLOSED|         DESOLC|
|       COMPLETE|       ETELPMOC|
+---------------+---------------+
only showing top 5 rows



## repeat

In [17]:
df.select(col("order_status"),repeat(df.order_status,2).alias("repeat_order_status")).show(5) # repating 2 times as we passed 2

+---------------+--------------------+
|   order_status| repeat_order_status|
+---------------+--------------------+
|         CLOSED|        CLOSEDCLOSED|
|PENDING_PAYMENT|PENDING_PAYMENTPE...|
|       COMPLETE|    COMPLETECOMPLETE|
|         CLOSED|        CLOSEDCLOSED|
|       COMPLETE|    COMPLETECOMPLETE|
+---------------+--------------------+
only showing top 5 rows



## Concat

In [18]:
df.withColumn("concated",concat(col("order_id"),lit(","),df.order_status)).show(5)

+--------+-------------------+-----------------+---------------+-----------------+
|order_id|         order_date|order_customer_id|   order_status|         concated|
+--------+-------------------+-----------------+---------------+-----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|         1,CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|2,PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|       3,COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|         4,CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|       5,COMPLETE|
+--------+-------------------+-----------------+---------------+-----------------+
only showing top 5 rows



## Concat_ws-- concate with separator

In [19]:
df.withColumn("concated",concat_ws("#",col("order_id"),df.order_status)).show(5)

+--------+-------------------+-----------------+---------------+-----------------+
|order_id|         order_date|order_customer_id|   order_status|         concated|
+--------+-------------------+-----------------+---------------+-----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|         1#CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|2#PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|       3#COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|         4#CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|       5#COMPLETE|
+--------+-------------------+-----------------+---------------+-----------------+
only showing top 5 rows



### Substring

In [20]:
df.withColumn("Year",substring(df.order_date,1,4)).show(5)

+--------+-------------------+-----------------+---------------+----+
|order_id|         order_date|order_customer_id|   order_status|Year|
+--------+-------------------+-----------------+---------------+----+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|2013|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|2013|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|2013|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|2013|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|2013|
+--------+-------------------+-----------------+---------------+----+
only showing top 5 rows



### Substring_index-- Works like scan in SAS

In [21]:
df.withColumn("Year",substring_index(df.order_date,"-",1)).show(5)

+--------+-------------------+-----------------+---------------+----+
|order_id|         order_date|order_customer_id|   order_status|Year|
+--------+-------------------+-----------------+---------------+----+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|2013|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|2013|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|2013|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|2013|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|2013|
+--------+-------------------+-----------------+---------------+----+
only showing top 5 rows



In [22]:
df.withColumn("Year",substring_index(df.order_date,"-",2)).show(5)

+--------+-------------------+-----------------+---------------+-------+
|order_id|         order_date|order_customer_id|   order_status|   Year|
+--------+-------------------+-----------------+---------------+-------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|2013-07|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|2013-07|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|2013-07|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|2013-07|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|2013-07|
+--------+-------------------+-----------------+---------------+-------+
only showing top 5 rows



In [24]:
df.withColumn("Year",substring(substring_index(df.order_date,"-",2),6,8)).show(5)

+--------+-------------------+-----------------+---------------+----+
|order_id|         order_date|order_customer_id|   order_status|Year|
+--------+-------------------+-----------------+---------------+----+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|  07|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|  07|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|  07|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|  07|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|  07|
+--------+-------------------+-----------------+---------------+----+
only showing top 5 rows



### instr--- Locate  position of first occurance of substring in given string

In [25]:
df.withColumn("PositionOfString",instr(df.order_status,"LO")).show(5)

+--------+-------------------+-----------------+---------------+----------------+
|order_id|         order_date|order_customer_id|   order_status|PositionOfString|
+--------+-------------------+-----------------+---------------+----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|               2|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|               0|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|               0|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|               2|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|               0|
+--------+-------------------+-----------------+---------------+----------------+
only showing top 5 rows



#### LOcate(sunstringTofind, String,Position(from where we want to start findling the word)

In [28]:
df.withColumn("PositionOfString",locate("E",df.order_status,3)).show(5)

+--------+-------------------+-----------------+---------------+----------------+
|order_id|         order_date|order_customer_id|   order_status|PositionOfString|
+--------+-------------------+-----------------+---------------+----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|               5|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|              13|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|               6|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|               5|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|               6|
+--------+-------------------+-----------------+---------------+----------------+
only showing top 5 rows



## Translate

In [33]:
df=spark.createDataFrame([("translate",)],("col",))
df.show()

+---------+
|      col|
+---------+
|translate|
+---------+



In [34]:
df.withColumn("translate",translate(df.col,"rnlt","123")).show()

+---------+---------+
|      col|translate|
+---------+---------+
|translate|  1a2s3ae|
+---------+---------+



## Here
* r is replaced by 1
* n is replaced by 2
* l is replaced by 3
* t is replaced by blank

## Overlay
* replace SQL with CORE

In [35]:
df=spark.createDataFrame([("SPARK_SQL","CORE")],("X","Y"))
df.show()

+---------+----+
|        X|   Y|
+---------+----+
|SPARK_SQL|CORE|
+---------+----+



In [37]:
df.withColumn("Overlayed_colmn",overlay(df.X,df.Y,7)).show()

+---------+----+---------------+
|        X|   Y|Overlayed_colmn|
+---------+----+---------------+
|SPARK_SQL|CORE|     SPARK_CORE|
+---------+----+---------------+

